In [ ]:
import sys
sys.path/home/surender/Downloads/carlaOld/PythonAPI/carla/dist/carla-0.9.9-py3.7-linux-x86_64.egg

In [ ]:
import carla


In [ ]:
!export OPENDD_DIR=~/Downloads/openDD
!export NGSIM_DIR=~/Downloads/NGSIM



In [ ]:
client =  carla.Client("localhost",2000)

In [ ]:
import os
os.environ["NGSIM_DIR"] = "/home/surender/Downloads/NGSIM"
os.environ["OPENDD_DIR"] = "/home/surender/Downloads/openDD"

import sys
#sys.path.append('/home/surender/Downloads/CARLA_0.9.9.4/PythonAPI/carla/dist')
import carla
import random
import argparse

from carla_real_traffic_scenarios.carla_maps import CarlaMaps
from carla_real_traffic_scenarios.ngsim import NGSimDatasets, DatasetMode
from carla_real_traffic_scenarios.ngsim.scenario import NGSimLaneChangeScenario
from carla_real_traffic_scenarios.opendd.scenario import OpenDDScenario
from carla_real_traffic_scenarios.reward import RewardType
from carla_real_traffic_scenarios.scenario import Scenario


from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

In [ ]:
NGSimDatasets.list()

In [ ]:
def parser_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--dataset", choices=["ngsim", "opendd"], default="opendd")
    parser.add_argument("--host", default="127.0.0.1")
    parser.add_argument("--port", default=2000, type=int)
    parser.add_argument("--num-episodes", default=10, type=int)
    args = parser.parse_args()
    return args


def prepare_ngsim_scenario(client: carla.Client) -> Scenario:
    data_dir = os.environ.get("NGSIM_DIR")
    #data_dir = os.listdir('/home/surender/Downloads/NGSIM')
    assert data_dir, "Path to the directory with NGSIM dataset is required"
    ngsim_map = NGSimDatasets.list()
    ngsim_dataset = random.choice(ngsim_map)
    client.load_world(ngsim_dataset.carla_map.level_path)
    return NGSimLaneChangeScenario(
        ngsim_dataset,
        dataset_mode=DatasetMode.TRAIN,
        data_dir=data_dir,
        reward_type=RewardType.DENSE,
        client=client,
    )


def prepare_opendd_scenario(client: carla.Client) -> Scenario:
    data_dir = os.environ.get("OPENDD_DIR")
    assert data_dir, "Path to the directory with openDD dataset is required"
    maps = ["rdb1", "rdb2", "rdb3", "rdb4", "rdb5", "rdb6", "rdb7"]
    map_name = random.choice(maps)
    carla_map = getattr(CarlaMaps, map_name.upper())
    client.load_world(carla_map.level_path)
    return OpenDDScenario(
        client,
        dataset_dir=data_dir,
        dataset_mode=DatasetMode.TRAIN,
        reward_type=RewardType.DENSE,
        place_name=map_name,
    )


def prepare_ego_vehicle(world: carla.World) -> carla.Actor:
    car_blueprint = world.get_blueprint_library().find("vehicle.audi.a2")

    # This will allow external scripts like manual_control.py or no_rendering_mode.py
    # from the official CARLA examples to take control over the ego agent
    car_blueprint.set_attribute("role_name", "hero")

    # spawn points doesnt matter - scenario sets up position in reset
    ego_vehicle = world.spawn_actor(
        car_blueprint, carla.Transform(carla.Location(0, 0, 500), carla.Rotation())
    )

    assert ego_vehicle is not None, "Ego vehicle could not be spawned"

    # Setup any car sensors you like, collect observations and then use them as input to your model
    return ego_vehicle

In [ ]:
if __name__ == "__main__":
    #args = parser_args()
    dataset = "ngsim"
    num_episodes = 5
    host = "localhost"
    port = 2000
    print(f"Trying to connect to CARLA server via {host}:{port}", end="...")
    client = carla.Client(host, port)
    client.set_timeout(60)

    if dataset == "ngsim":
        scenario = prepare_ngsim_scenario(client)
    elif dataset == "opendd":
        scenario = prepare_opendd_scenario(client)
    
    world = client.get_world()
    spectator = world.get_spectator()
    ego_vehicle = prepare_ego_vehicle(world)
    scenario.reset(ego_vehicle)

    # OpenAI Gym interface:
    for ep_idx in range(num_episodes):
        print(f"Running episode {ep_idx+1}/{num_episodes}")

        # Ego vehicle replaces one of the real-world vehicles
        scenario.reset(ego_vehicle)
        done = False
        total_reward = 0
        while not done:
            
            # Read sensors, use policy to generate action and apply it to control ego agent
            # ego_vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=-1.0))

            chauffeur_cmd, reward, done, info = scenario.step(ego_vehicle)
            total_reward += reward
            print(f"Step: command={chauffeur_cmd.name}, total_reward={total_reward}")
            world.tick()
            print(len(world.get_actors()))
            # Server camera follows ego
            topdown_view = carla.Transform(
                ego_vehicle.get_transform().location + carla.Vector3D(x=0, y=0, z=30),
                carla.Rotation(pitch=-90),
            )
            spectator.set_transform(topdown_view)

    scenario.close()

In [ ]:
host = "localhost"
port = 2000
client = carla.Client(host,port)

In [ ]:
scenario = prepare_ngsim_scenario(client)

In [ ]:
from carla_birdeye_view import BirdViewProducer, BirdViewCropType, PixelDimensions

birdview_producer = BirdViewProducer(
    client,  # carla.Client
    target_size=PixelDimensions(width=150, height=336),
    pixels_per_meter=4,
    crop_type=BirdViewCropType.FRONT_AND_REAR_AREA
)

In [ ]:
from PIL import Image

In [ ]:
world = client.get_world()
spectator = world.get_spectator()
ego_vehicle = prepare_ego_vehicle(world)
scenario.reset(ego_vehicle)
world.tick()

In [ ]:
birdview = birdview_producer.produce(
    agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
)

In [ ]:
cv2.fr

In [ ]:
from IPython.display import clear_output, Image, display, HTML

In [ ]:
def arrayShow(imageArray):
    ret, png = cv2.imencode('.jpg', imageArray)
    encoded = base64.b64encode(png)
    return Image(data=encoded.decode('ascii'))

In [ ]:
world.tick()

In [ ]:
rgb = BirdViewProducer.as_rgb(birdview)
img = Image.fromarray(rgb, 'RGB')
img.save('my.png')
img.show()

In [ ]:
import cv2

In [ ]:
out = cv2.VideoWriter('project1.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, (150,336))

In [ ]:
for i in range(100):
    birdview = birdview_producer.produce(
    agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
    )
    rgb = BirdViewProducer.as_rgb(birdview)
    world.tick()
    out.write(rgb)
out.release()

In [ ]:
done = False
def gen():
    global reward
    i = 0
    while not done:
        i += 1
        yield i
        

In [ ]:


#highgui.cvNamedWindow('mahMovie')

for i in range(10):
    birdview = birdview_producer.produce(
    agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
    )
    rgb = BirdViewProducer.as_rgb(birdview)
    
    cv2.imshow('mahMovie', rgb)
    world.tick()

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
%matplotlib tk

import cv2
import matplotlib.pyplot as plt







for i in range(100):
    #fig.clear()
    birdview = birdview_producer.produce(
    agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
    )
    rgb = BirdViewProducer.as_rgb(birdview)
    cv2.imshow('Frame',rgb)
    if cv2.waitKey(25) & 0xFF == ord('q'):

          break

    #plt.draw()
    world.tick()
cv2.destroyAllWindows()


In [ ]:
cv2.destroyAllWindows()


In [ ]:
q%matplotlib tk
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
import threading
import time

num_ep = 10
ep = 0
fig = plt.figure()

def run_animations():
    global done
    done=False
    anim = animation.FuncAnimation(fig, updatefig, gen,repeat = False)
    
def wait_for_done():
    global done
    while not done:
        continue
p = threading.Thread(target=run_animations)
q = threading.Thread(target=wait_for_done)
def updatefig(i):
    global done
    fig.clear()
    birdview = birdview_producer.produce(
    agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
    )
    rgb = BirdViewProducer.as_rgb(birdview)
    #print(rgb.shape)
    #img = Image.fromarray(rgb, 'RGB')
    
    p = plt.imshow(rgb)
    
    plt.draw()
    
    if i>50:
        print("DONE")
        done = True
        ep+=1
        scenario.reset(ego_vehicle)
        plt.close()
    world.tick()


for i in range(5):
    p.start()
    q.start()
    print("NOW WAIT")
    p.join()
    q.join()
    


#anim.save("/tmp/test.mp4", fps=1)

In [ ]:
fig = plt.figure()

for i in range(50):
    fig.clear()
    birdview = birdview_producer.produce(
    agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
    )
    rgb = BirdViewProducer.as_rgb(birdview)
    p = plt.imshow(rgb)
    plt.draw()
    world.tick()

In [ ]:
scenario.action_space

In [ ]:
cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp.set_attribute("image_size_x",str(320))
cam_bp.set_attribute("image_size_y",str(320))
cam_bp.set_attribute("fov",str(150))
cam_location = carla.Location(-2,0,1)
cam_rotation = carla.Rotation(0,180,0)
cam_transform = carla.Transform(cam_location,cam_rotation)
ego_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_cam.listen(lambda image: image.save_to_disk("hi.jpg"))

In [ ]:
geo_bp = world.get_blueprint_library().find('sensor.other.gnss')
geo_loc = carla.Location(0,0,0)
geo_rot = carla.Rotation(0,0,0)
geo_transform = carla.Transform(geo_loc,geo_rot)
ego_gnss = world.spawn_actor(geo_bp,geo_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_gnss.listen(lambda dat: print(dat))

In [ ]:
def itr(dat):
    print("HERE")
    len(dat)

In [ ]:
rdr_bp = world.get_blueprint_library().find('sensor.other.radar')
rdr_loc = carla.Location(0,0,0)
rdr_rot = carla.Rotation(0,0,0)
rdr_transform = carla.Transform(rdr_loc,rdr_rot)
ego_rdr = world.spawn_actor(rdr_bp,rdr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_rdr.listen(lambda dat: print(len(dat)))

In [ ]:
def print_all_lidar(dat):
    print(len(dat))
    for i in dat:
        print(i)

In [ ]:
ldr_bp = world.get_blueprint_library().find('sensor.other.radar')
ldr_loc = carla.Location(0,0,0)
ldr_rot = carla.Rotation(0,0,0)
ldr_bp.set_attribute("range",'3.0')


ldr_transform = carla.Transform(ldr_loc,ldr_rot)
ego_ldr = world.spawn_actor(ldr_bp,ldr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_ldr.listen(lambda dat: print_all_lidar(dat))

In [ ]:
rdr_bp = world.get_blueprint_library().find('sensor.other.radar')
rdr_loc = carla.Location(0,0,0)
rdr_rot = carla.Rotation(0,180,0)
rdr_transform = carla.Transform(rdr_loc,rdr_rot)
ego_rdr = world.spawn_actor(rdr_bp,rdr_transform, attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_rdr.listen(lambda dat: print("HI"))

In [ ]:
world.tick()

In [ ]:
ego_vehicle

In [ ]:
for i in range(len(world.get_blueprint_library())):
    print(world.get_blueprint_library()[i])

In [ ]:
len(world.get_actors())

In [ ]:
ego_cam

In [ ]:
!export NGSIM_DIR=/home/surender/Downloads/NGSIM

In [ ]:
!echo $NGSIM_DIR 

In [ ]:
os.environ.get("$NSGIM_DIR")

In [ ]:
!export HI=/home/surender

In [ ]:
os.environ.get("/home/sure")

In [ ]:
os.listdir('/home/surender/Downloads/NGSIM')

In [ ]:
!echo $OPENDD_DIR

In [ ]:
import numpy as np

In [ ]:
np.clip(0.3,0,1)

In [ ]:
set_var = []

In [ ]:
from keras.layers import Input
from keras.models import Model

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=True)
model = Model(inputs=base_model.input,outputs=base_model.get_layer('avg_pool').output)

In [ ]:
print(model.summary())

In [ ]:
final_features  = 0
final_features1 = 0

In [ ]:
import numpy as np
from PIL import Image
def get_features(img):
    global final_features
    array = np.frombuffer(img.raw_data, dtype=np.dtype("uint8")) 
    array = np.reshape(array, (img.height, img.width, 4)) # RGBA format
    array = array[:, :, :3] #  Take only RGB
    
    
    img = Image.fromarray(array)
    img = img.resize((299, 299), Image.ANTIALIAS)
    input_data = np.array(img)
    #print("GOOD")
    x = np.expand_dims(input_data, axis=0)
    x = preprocess_input(x)
    #print("FINE")
    features = model.predict(x) 
    #print("WHY")
    features = features[0]
    #print("WORKS?")
    final_features = features
    
def get_features1(img):
    global final_features1
    array = np.frombuffer(img.raw_data, dtype=np.dtype("uint8")) 
    array = np.reshape(array, (img.height, img.width, 4)) # RGBA format
    array = array[:, :, :3] #  Take only RGB
    img = Image.fromarray(array)
    img = img.resize((299, 299), Image.ANTIALIAS)
    input_data = np.array(img)
    #print("NO")
    x = np.expand_dims(input_data, axis=0)
    x = preprocess_input(x)
    #print("MAYBE")
    features = model.predict(x) 
    #print("WHI NOS")
    features = features[0]
    
    final_features1 = features

In [ ]:
cam_bp = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp.set_attribute("image_size_x",str(320))
cam_bp.set_attribute("image_size_y",str(320))
cam_bp.set_attribute("fov",str(100))
cam_location = carla.Location(2,0,1)
cam_rotation = carla.Rotation(0,0,0)
cam_transform = carla.Transform(cam_location,cam_rotation)
ego_cam = world.spawn_actor(cam_bp,cam_transform,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_cam.listen(lambda image: get_features(image))

In [ ]:
cam_bp1 = world.get_blueprint_library().find('sensor.camera.rgb')
cam_bp1.set_attribute("image_size_x",str(320))
cam_bp1.set_attribute("image_size_y",str(320))
cam_bp1.set_attribute("fov",str(100))
cam_location1 = carla.Location(-2,0,1)
cam_rotation1 = carla.Rotation(0,180,0)
cam_transform1 = carla.Transform(cam_location1,cam_rotation1)
ego_cam1 = world.spawn_actor(cam_bp1,cam_transform1,attach_to=ego_vehicle, attachment_type=carla.AttachmentType.Rigid)
ego_cam1.listen(lambda image1: get_features1(image1))

In [ ]:
world.tick()

In [ ]:
final_features.shape

In [ ]:
final_features1.shape

In [ ]:
(final_features + final_features1).shape

In [1]:
a = np.append(final_features, final_features1)

NameError: name 'np' is not defined

In [ ]:
1536+1536

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
Image

In [ ]:
birdview = birdview_producer.produce(
agent_vehicle=ego_vehicle  # carla.Actor (spawned vehicle)
)
rgb = BirdViewProducer.as_rgb(birdview)
cv2.imshow('Frame',rgb)


In [ ]:
cv2.destroyAllWindows()

In [ ]:
net = InceptionResNetV2(include_top=True,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(299,299,3))

In [ ]:
print(net.summary())

In [4]:
import numpy as np

In [5]:
a = np.random.rand(5,)

In [11]:
c = a.reshape(a.shape[0])

In [14]:
list(a)

[0.29341825691336976,
 0.8793488004025491,
 0.0666725049362531,
 0.2084351205887519,
 0.742487181416361]